In [39]:
import re
from konlpy.tag import Okt
import pandas as pd
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

# 불러올 파일들의 공통 부분과 도시 목록 정의
cities = ['파리', '오사카', '방콕', '뉴욕']
categories = ['맛집', '관광', '핫플', '숙소', '쇼핑', '여행']

# Load SBERT model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # 다국어 지원

# Konlpy의 Okt 객체 생성
okt = Okt()

# 모든 도시와 카테고리 파일을 한꺼번에 불러오는 코드
for city in cities:
    for category in categories:

        # 조사나 접속사, 이모티콘 등을 제거하는 함수
        def clean_korean_text(text):
            if pd.isna(text):  # NaN 값일 경우 빈 문자열로 처리
                return ''
            
            # 1. 이모티콘과 특수문자 제거 (정규 표현식 사용)
            text = re.sub(r'[^\w\s]', '', text)  # 이모티콘, 특수 문자 제거
            
            # 2. 형태소 분석 후 조사, 접속사 제거
            morphs = okt.pos(text, norm=True, stem=True)
            
            # 3. 조사와 접속사 필터링 (조사(Josa), 접속사(Conjunction) 태그 제외)
            cleaned_text = ' '.join(word for word, tag in morphs if tag not in ['Josa', 'Conjunction', 'Adjective', 'Adverb', 'Eomi', 'Determiner', 'Exclamation', 'Suffix', 'Verb', 'KoreanParticle'])    
        
            return cleaned_text
        
        # 청킹 함수: 명사(Noun), 형용사(Adjective) 등을 추출해 묶는 방식
        def chunk_korean_text(text):
            english_words = re.findall(r'[a-zA-Z]+', text)
            
            morphs = okt.pos(text, norm=True, stem=True)
            
            filtered_words = [word for word, pos in morphs if pos in ['Noun', 'Alpha']]  # 명사와 영어 단어만 남김
            
            # 영어 단어 다시 추가
            filtered_words.extend(english_words)
            
            return ' '.join(filtered_words)
        
        # Function to get embedding for a single text
        def get_embedding(text):
            return model.encode([text])[0]  # Returns the embedding as a numpy array
        
        # Cache to store embeddings of hashtags
        embeddings_cache = {}
        
        # Function to get relevant hashtags for each post
        def get_relevant_hashtags(tags):
            if len(tags) == 0:
                return []
            
            # For new hashtags, compute embeddings and store them in the cache
            new_tags = [tag for tag in tags if tag not in embeddings_cache]
            if new_tags:
                new_embeddings = model.encode(new_tags)  # Batch encode new tags
                embeddings_cache.update({new_tags[i]: new_embeddings[i] for i in range(len(new_tags))})
            
            # Use cached embeddings for all tags
            tag_embeddings = [embeddings_cache[tag] for tag in tags]
            
            # Compute cosine similarity between the keyword and all tag embeddings
            similarities = [cosine_similarity(keyword_embedding, np.array(tag_embedding).reshape(1, -1))[0][0] for tag_embedding in tag_embeddings]
            
            # Filter hashtags with similarity above a certain threshold (e.g., 0.5)
            relevant_tags = [tags[i] for i in range(len(tags)) if similarities[i] > 0.5]
            
            return relevant_tags
        
        def clean_tags(text):
            # 한글, 영어, 숫자, '#'을 제외한 나머지 문자 제거
            return re.findall(r'#[A-Za-z0-9가-힣]+', text)
        
        # 파일 읽기
        file_path = f"./인스타그램 데이터/{city}{category}_crawling.xlsx"  # 파일 경로를 적절히 변경해주세요.
        df = pd.read_excel(file_path)
        print(f" '{city}{category}' 파일불러오기 완료.")

        # Get the embedding for the keyword '뉴욕관광'
        keyword_embedding = get_embedding(f'{city}{category}')
        keyword_embedding = np.array(keyword_embedding).reshape(1, -1)  # Reshape to 2D for cosine similarity
        
        # 1. content 컬럼에서 조사, 접속사, 이모티콘 제거 적용
        df['cleaned_content'] = df['content'].apply(lambda x: clean_korean_text(x))
        
        # 2. 조사, 접속사 제거된 텍스트에서 청킹 적용 (명사, 형용사, 동사만 남김)
        df['chunked_content'] = df['cleaned_content'].apply(lambda x: chunk_korean_text(x))
        
        df['cleaned_tags'] = df['tags'].apply(clean_tags)
        
        # Activate tqdm for pandas and apply the function to each post, with caching
        tqdm.pandas()
        df['relevant_hashtags'] = df['cleaned_tags'].progress_apply(lambda tags: get_relevant_hashtags(tags))
        
         # 3. CSV 파일로 저장 (필터링된 청킹된 본문 및 해시태그)
        output_file = f'{city}{category}_인스타그램_최종.csv'
        df[['chunked_content', 'place', 'relevant_hashtags']].to_csv(output_file, index=False, encoding='utf-8-sig')
        
        print(f" '{output_file}' 파일로 저장 완료.")
        print()

 '파리맛집' 파일불러오기 완료.


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:19<00:00, 26.15it/s]


 '파리맛집_인스타그램_최종.csv' 파일로 저장 완료.

 '파리관광' 파일불러오기 완료.


100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:04<00:00, 117.77it/s]


 '파리관광_인스타그램_최종.csv' 파일로 저장 완료.

 '파리핫플' 파일불러오기 완료.


...

 '뉴욕숙소_인스타그램_최종.csv' 파일로 저장 완료.

 '뉴욕쇼핑' 파일불러오기 완료.


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:29<00:00, 16.82it/s]


 '뉴욕쇼핑_인스타그램_최종.csv' 파일로 저장 완료.

 '뉴욕여행' 파일불러오기 완료.


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:16<00:00, 29.76it/s]

 '뉴욕여행_인스타그램_최종.csv' 파일로 저장 완료.

